In [1]:
from benchmark import *
%matplotlib inline

In [2]:
helpers = 'pdb'
mcss = '../../../mcss_sizes.pkl'

sp = 'stats21'
xp = 'stats20'

def results(mcss_func, name, aggregate):
    data = pd.concat(load(version, helpers, mcss) for version in [sp, xp])
    data = add_correct(data, thresh = 2.0)
    data = data[(data.index.get_level_values('protein') != 'A2AR')]
    data = data[mcss_func(data.mcss)]

    ligand = data.xs((helpers, 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                      level=('helpers', 'mode', 'n_ligs', 'features', 'alpha')).filter(regex='rmsd|correct')
    ligand = filter_to_ubiquitous_ligands(ligand)
    target = ligand.groupby(level=list(range(len(ligand.index.levels)-1))).mean().dropna()
    
    if aggregate == 'target':
        family = target.groupby(level=list(range(len(target.index.levels)-1))).mean()
    elif aggregate == 'ligand':
        family = ligand.groupby(level=list(range(len(ligand.index.levels)-2))).mean()

    target = pd.merge(target.loc[sp], target.loc[xp], left_index=True, right_index=True)
    family = pd.merge(family.loc[sp], family.loc[xp], left_index=True, right_index=True)
    
    
    

    family['SP Top'] = family['glide_correct_x']
    family['SP ComBind'] = family['combind_correct_x']
    family['SP Best'] = family['best_correct_x']
    family['XP Top'] = family['glide_correct_y']
    family['XP ComBind'] = family['combind_correct_y']
    family['XP Best'] = family['best_correct_y']
    family = family.filter(regex='P')
    
    family['version'] = name
    family.reset_index(inplace=True)
    family.set_index(['version', 'family'], inplace = True)
    return drug_average(family)

In [3]:
pd.concat([results(lambda x: x > -1, 'All', 'target'),
           results(lambda x: x >= 0.5, 'Shared scaffold', 'target'),
           results(lambda x: x < 0.5, 'No shared scaffold', 'target')])

,SP Top,SP ComBind,SP Best,XP Top,XP ComBind,XP Best
version,,,,,,
All,0.440596,0.564133,0.803741,0.525265,0.561086,0.659842
Shared scaffold,0.466675,0.578302,0.876302,0.612218,0.759300,0.804294
No shared scaffold,0.410593,0.542547,0.787777,0.430961,0.475231,0.584048


In [4]:
pd.concat([results(lambda x: x > -1, 'All', 'ligand'),
           results(lambda x: x >= 0.5, 'Shared scaffold', 'ligand'),
           results(lambda x: x < 0.5, 'No shared scaffold', 'ligand')])

,SP Top,SP ComBind,SP Best,XP Top,XP ComBind,XP Best
version,,,,,,
All,0.456614,0.595907,0.807639,0.515893,0.566911,0.681468
Shared scaffold,0.562691,0.731723,0.862450,0.713093,0.776031,0.822499
No shared scaffold,0.408640,0.534663,0.765993,0.432684,0.476204,0.614611


# Number of ligands for each target

In [6]:
data = pd.concat(load(version, helpers, mcss) for version in [sp, xp])
data = add_correct(data, thresh = 2.0)

all_ligands = data.xs(('stats21', 'pdb', 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                  level=('version', 'helpers', 'mode', 'n_ligs', 'features', 'alpha'))

diverse = data[data.mcss < 0.5].xs(('stats21', 'pdb', 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                  level=('version', 'helpers', 'mode', 'n_ligs', 'features', 'alpha'))

diverse_correct = data[(data.mcss < 0.5) & data.best_correct].xs(
                    ('stats21', 'pdb', 'standard', 0, 'mcss_contact_hbond_sb', 1.0),
                     level=('version', 'helpers', 'mode', 'n_ligs', 'features', 'alpha'))

df = pd.DataFrame()
df['all'] = all_ligands.groupby(level='protein').count().iloc[:, 0]
df['diverse'] = diverse.groupby(level='protein').count().iloc[:, 0]
df['diverse_correct'] = diverse_correct.groupby(level='protein').count().iloc[:, 0]
df

,all,diverse,diverse_correct
protein,,,
5HT2B,5,5.0,5.0
A2AR,17,15.0,10.0
AR,19,12.0,10.0
B1AR,11,6.0,6.0
B2AR,7,4.0,4.0
BACE1,20,19.0,6.0
BRAF,20,20.0,14.0
BRD4,16,13.0,7.0
CDK2,20,20.0,17.0


In [7]:
df.sum()

all                524.0
diverse            423.0
diverse_correct    294.0
dtype: float64